## Results Processing

In [10]:
import os 
import pandas as pd
from langfuse import Langfuse
langfuse = Langfuse()

In [14]:
RESULTS_FOLDER = "../../results/"

In [5]:
def langfuse_run_ids(dataset_name, dataset_run_name):
    dataset_run = langfuse.get_dataset_run(
    dataset_name=dataset_name, dataset_run_name=dataset_run_name
    )
    trace_ids = [run_item.trace_id for run_item in dataset_run.dataset_run_items]
    return trace_ids

In [12]:
def open_results(dataset):
    return pd.read_csv(os.path.join(RESULTS_FOLDER, dataset))

In [20]:
trace_ids = langfuse_run_ids("RAGHPO", "2025-05-01_18-19-50")
results = open_results("rag_hpo_ensembleretriever.csv")
results = results[results.id.isin(trace_ids)]

In [45]:
results[results.id.isin(results[results.duplicated("input", keep=False)].id)].sort_values("input")

,id,projectId,timestamp,tags,bookmarked,name,release,version,userId,environment,...,inputTokens,outputTokens,totalTokens,f1,jaccard_similarity,precision,r_precision,r_recall,recall,semantic similarity
59,cda1abd4-649e-45d4-9d0f-f1c9e3d64431,cm9vlvsif0006pf07xcychmbe,2025-05-02T08:37:26.825Z,[],False,custom_chain,NaN,NaN,NaN,default,...,27096,10216,37312,[0.2],[0.125],[0.125],[0.014184397163120567],[0.5],[0.5],[0.18133299869227826]
42,6ba4353a-5b37-43af-bd84-46d6e7c3e020,cm9vlvsif0006pf07xcychmbe,2025-05-02T08:38:38.522Z,[],False,custom_chain,NaN,NaN,NaN,default,...,19772,8231,28003,[0.13],[0.08333333333333333],[0.09090909090909091],[0.009615384615384616],[0.25],[0.25],[0.22483112871131647]
70,4aa19563-3cee-4bc7-9663-f3a188cea9fb,cm9vlvsif0006pf07xcychmbe,2025-05-02T08:36:36.431Z,[],False,custom_chain,NaN,NaN,NaN,default,...,5096,3224,8320,[0.57],[0.4],[0.5],[0.10714285714285714],[1],[0.6666666666666666],[0.7252138218979445]
90,83a5b342-dcc6-488f-8c07-81ee2d17e1ab,cm9vlvsif0006pf07xcychmbe,2025-05-02T08:35:28.633Z,[],False,custom_chain,NaN,NaN,NaN,default,...,5317,2563,7880,[0.57],[0.4],[0.5],[0.11538461538461539],[1],[0.6666666666666666],[0.7252138218979445]
3,077a18b3-cb3f-47a4-a86e-98c02bdd589c,cm9vlvsif0006pf07xcychmbe,2025-05-02T08:42:02.700Z,[],False,custom_chain,NaN,NaN,NaN,default,...,12374,5378,17752,[0.66],[0.45454545454545453],[0.5454545454545454],[0.08064516129032258],[0.8333333333333334],[0.8333333333333334],[0.6009701331376598]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,6bfd9919-82ba-4932-9ff1-e34efd848afa,cm9vlvsif0006pf07xcychmbe,2025-05-02T08:39:41.175Z,[],False,custom_chain,NaN,NaN,NaN,default,...,26069,6920,32989,[0.67],[0.5],[0.6875],[0.13636363636363635],[0.8823529411764706],[0.6470588235294118],[0.6035362994875284]
54,88d1d53d-86e2-4db5-bb0e-fd68f3eda677,cm9vlvsif0006pf07xcychmbe,2025-05-02T08:37:46.413Z,[],False,custom_chain,NaN,NaN,NaN,default,...,33127,9123,42250,[0.66],[0.5217391304347826],[0.65],[0.10909090909090909],[0.6666666666666666],[0.6666666666666666],[0.6077007164307483]
72,de02908f-a039-4e1a-9e77-02164f64ba95,cm9vlvsif0006pf07xcychmbe,2025-05-02T08:36:29.695Z,[],False,custom_chain,NaN,NaN,NaN,default,...,30311,10312,40623,[0.71],[0.5652173913043478],[0.7],[0.12096774193548387],[0.8333333333333334],[0.7222222222222222],[0.5857322855641125]
16,775af4ef-2193-4af6-af5b-a69725ca9c03,cm9vlvsif0006pf07xcychmbe,2025-05-02T08:40:23.893Z,[],False,custom_chain,NaN,NaN,NaN,default,...,20365,8939,29304,[0.38],[0.25],[0.38461538461538464],[0.0898876404494382],[0.6153846153846154],[0.38461538461538464],[0.5074074012377019]
